<h1> I AM RUNNING ON NORMALIZED DATA


In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GroupKFold
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import precision_score, recall_score, f1_score, balanced_accuracy_score
import pickle
from collections import defaultdict
from sklearn.utils import shuffle
import os
import cv2
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import pandas as pd

from torchinfo import summary
from sklearn.metrics import confusion_matrix
import seaborn as sns



In [2]:
import numpy as np
import os

# Define the folder path
folder_path = r"D:\PYTHONIG\newwindow\NOTEBOOKS_2025\aprilmay2025\data\numpy\melnormalized\concatenatedspectrograms"

# Load the numpy files into the respective arrays with the correct capitalized naming
eeg_fold_1 = np.load(os.path.join(folder_path, 'MEL_DATA_FOLD_fold_1.npy'))
labels_fold_1 = np.load(os.path.join(folder_path, 'MEL_LABELS_FOLD_fold_1.npy'))
patients_fold_1 = np.load(os.path.join(folder_path, 'MEL_PATIENTS_FOLD_fold_1.npy'))

eeg_fold_2 = np.load(os.path.join(folder_path, 'MEL_DATA_FOLD_fold_2.npy'))
labels_fold_2 = np.load(os.path.join(folder_path, 'MEL_LABELS_FOLD_fold_2.npy'))
patients_fold_2 = np.load(os.path.join(folder_path, 'MEL_PATIENTS_FOLD_fold_2.npy'))

eeg_fold_3 = np.load(os.path.join(folder_path, 'MEL_DATA_FOLD_fold_3.npy'))
labels_fold_3 = np.load(os.path.join(folder_path, 'MEL_LABELS_FOLD_fold_3.npy'))
patients_fold_3 = np.load(os.path.join(folder_path, 'MEL_PATIENTS_FOLD_fold_3.npy'))

eeg_fold_4 = np.load(os.path.join(folder_path, 'MEL_DATA_FOLD_fold_4.npy'))
labels_fold_4 = np.load(os.path.join(folder_path, 'MEL_LABELS_FOLD_fold_4.npy'))
patients_fold_4 = np.load(os.path.join(folder_path, 'MEL_PATIENTS_FOLD_fold_4.npy'))

eeg_fold_5 = np.load(os.path.join(folder_path, 'MEL_DATA_FOLD_fold_5.npy'))
labels_fold_5 = np.load(os.path.join(folder_path, 'MEL_LABELS_FOLD_fold_5.npy'))
patients_fold_5 = np.load(os.path.join(folder_path, 'MEL_PATIENTS_FOLD_fold_5.npy'))

eeg_folds = [eeg_fold_1, eeg_fold_2, eeg_fold_3, eeg_fold_4, eeg_fold_5]
labels_folds = [labels_fold_1, labels_fold_2, labels_fold_3, labels_fold_4, labels_fold_5]
patients_folds = [patients_fold_1, patients_fold_2, patients_fold_3, patients_fold_4, patients_fold_5]

for i in range(len(eeg_folds)):
    eeg_folds[i] = eeg_folds[i].astype(np.float16)

In [3]:
def data_balancer(data, labels, factor):
    # Count the number of samples in each class
    num_class_0 = np.sum(labels == 0)
    num_class_1 = np.sum(labels == 1)
    num_class_2 = np.sum(labels == 2)

    # Find the minimum number of samples across all classes
    min_samples = min(num_class_0, num_class_1, num_class_2)

    # Calculate the number of samples to take from each class
    samples_per_class = min_samples // factor

    # Randomly sample 'samples_per_class' from each class
    class_0_indices = np.random.choice(np.where(labels == 0)[0], samples_per_class, replace=False)
    class_1_indices = np.random.choice(np.where(labels == 1)[0], samples_per_class, replace=False)
    class_2_indices = np.random.choice(np.where(labels == 2)[0], samples_per_class, replace=False)

    # Combine balanced indices
    balanced_indices = np.concatenate((class_0_indices, class_1_indices, class_2_indices))

    # Shuffle the balanced indices
    np.random.shuffle(balanced_indices)

    # Create balanced training data and labels
    balanced_data = data[balanced_indices]
    balanced_labels = labels[balanced_indices]

    return balanced_data, balanced_labels

In [4]:
class EarlyStopping:
    def __init__(self, patience=5):
        """
        Initializes the early stopping mechanism based on divergence detection.

        Args:
            patience (int): Number of consecutive epochs with increasing validation loss
                            before stopping.
        """
        self.patience = patience
        self.best_loss = None
        self.counter = 0
        self.early_stop = False
        self.best_model_state = None

    def __call__(self, val_loss, model):
        """
        Checks if the validation loss is diverging and updates the state accordingly.

        Args:
            val_loss (float): Current epoch's validation loss.
            model (torch.nn.Module): The model being trained.
        """
        if self.best_loss is None or val_loss < self.best_loss:
            # Improvement detected
            self.best_loss = val_loss
            self.best_model_state = model.state_dict()
            self.counter = 0
        else:
            # Validation loss increased
            self.counter += 1
            if self.counter >= self.patience:
                print(f"Divergence detected. Stopping training after {self.counter} epochs.")
                self.early_stop = True

    def load_best_model(self, model):
        """
        Restores the model to the state with the lowest validation loss.

        Args:
            model (torch.nn.Module): The model to restore.
        """
        model.load_state_dict(self.best_model_state)




In [5]:
import torch
import torch.nn as nn

debug_mode_flag = False
# Set the device to GPU if available, otherwise use CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#set random seed for reproducibility
torch.manual_seed(42)
debug_mode_flag = False



class CustomCnn(nn.Module):
    def __init__(self, debug_mode_flag=False):
        super().__init__()
        self.debug_mode_flag = debug_mode_flag
        
        self.block_1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=16, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # Reduces spatial size
        )
        
        self.block_2 = nn.Sequential(
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)  # Further reduces spatial size
        )

        # Global Average Pooling to reduce spatial dimensions 
        self.global_avg_pool = nn.AdaptiveAvgPool2d((8, 8))  # Keeps a manageable seq_len
        self.flatten = nn.Flatten(start_dim=2)  # Keeps batch & channel dims

    def forward(self, x):
        if self.debug_mode_flag: print(f"Input shape: {x.shape}")
        
        x = self.block_1(x)
        if self.debug_mode_flag: print(f"Block 1 shape: {x.shape}")
        
        x = self.block_2(x)
        if self.debug_mode_flag: print(f"Block 2 shape: {x.shape}")
        
        x = self.global_avg_pool(x)  # (batch, 128, 8, 8)
        if self.debug_mode_flag: print(f"Global Avg Pool shape: {x.shape}")

        # x = self.flatten(x)  # (batch, 128, 64)
        # if self.debug_mode_flag: print(f"Flattened shape (Transformer Input): {x.shape}")
        
        return x

    
class TransformerEncoder(nn.Module):
    def __init__(self, embed_dim, num_heads, ff_dim, dropout_rate=0.1):
        super(TransformerEncoder, self).__init__()
        self.att = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads)
        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, ff_dim),
            nn.ReLU(),
            nn.Linear(ff_dim, embed_dim)
        )
        self.layernorm1 = nn.LayerNorm(embed_dim)
        self.layernorm2 = nn.LayerNorm(embed_dim)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.dropout2 = nn.Dropout(dropout_rate)

    def forward(self, x):
        attn_output, _ = self.att(x, x, x)
        attn_output = self.dropout1(attn_output)
        out1 = self.layernorm1(x + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output)
        return self.layernorm2(out1 + ffn_output)
    
class TRANS_CNN(nn.Module):
    def __init__(self, input_shape, num_classes, embed_dim=512, num_heads=2, ff_dim=64, num_transformer_blocks=4):
        
        super(TRANS_CNN,self).__init__()
        
        self.num_transformer_blocks = num_transformer_blocks
        self.cnn_extractor = CustomCnn()
        
        self.projection = nn.Linear(512, embed_dim)
        
        self.encoder = nn.ModuleList([
            TransformerEncoder(embed_dim,num_heads,ff_dim) for _ in range(num_transformer_blocks)
        ])
        
        self.global_avg_pool = nn.AdaptiveAvgPool1d(1)
        self.dropout = nn.Dropout(0.1)
        
        self.precls = nn.Linear(embed_dim,embed_dim)
        self.precls2 = nn.Linear(embed_dim,embed_dim)
        self.precls3 = nn.Linear(embed_dim,embed_dim//4)
        
        self.clf = nn.Linear(embed_dim//4,num_classes)
        
        
    def forward(self,x):
        
        x = self.cnn_extractor(x)
        if debug_mode_flag: print(f"x shape after cnn extraction = {x.shape}")
        
        B,C,H,W = x.shape
        
        x = x.view(B,H*W,C)
        if debug_mode_flag: print(f"x shape after changing view= {x.shape}")
        
        # x = self.projection(x)
        # if debug_mode_flag: print(f"x shape after projection= {x.shape}")
        
        for encoderblock in self.encoder:
            x = encoderblock(x)
            
        if debug_mode_flag: print(f"x shape after passing thru encoder= {x.shape}")
        
        x = x.permute(1,0,2)
        if debug_mode_flag: print(f"x shape after permuting{x.shape}")
        
        x = self.precls3(x)
        if debug_mode_flag: print(f"precls3 {x.shape}")
        
        x = self.dropout(x)
        
        x = x.mean(dim=0)  # Global average pooling over sequence (9 tokens → 1 token)
        if debug_mode_flag: print(f"x shape after average pooling {x.shape}")

        x = self.clf(x)  #they see me rolling
        if debug_mode_flag: print(f"cls {x.shape}")
        
        
        return x

In [6]:
from torchinfo import summary
model = TRANS_CNN(input_shape=(224,224,20),num_classes=3,num_transformer_blocks=4,embed_dim=320)
summary(model)

Layer (type:depth-idx)                                       Param #
TRANS_CNN                                                    --
├─CustomCnn: 1-1                                             --
│    └─Sequential: 2-1                                       --
│    │    └─Conv2d: 3-1                                      448
│    │    └─BatchNorm2d: 3-2                                 32
│    │    └─ReLU: 3-3                                        --
│    │    └─Conv2d: 3-4                                      4,640
│    │    └─BatchNorm2d: 3-5                                 64
│    │    └─ReLU: 3-6                                        --
│    │    └─MaxPool2d: 3-7                                   --
│    └─Sequential: 2-2                                       --
│    │    └─Conv2d: 3-8                                      18,496
│    │    └─BatchNorm2d: 3-9                                 128
│    │    └─ReLU: 3-10                                       --
│    │    └─Conv2d: 3-11  

In [7]:
torch.manual_seed(42)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# pca = sklearn.decomposition.PCA(3)
epochs = 100


fold_indices = np.arange(5)
fold_indices = np.random.permutation(fold_indices)
val_fold_indices = np.roll(fold_indices, 1)

input_shape = (3,224,224)
num_classes = 3


In [ ]:
import optuna
import torch
import torch.optim as optim
import torch.nn as nn
import numpy as np
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import balanced_accuracy_score

# import warnings
# warnings.filterwarnings("ignore", category=UserWarning, message=".*step is already reported.*")

#set random seed for reproducibility
torch.manual_seed(42)
np.random.seed(42)
debug_mode_flag = False

def objective(trial):
    # Hyperparameter suggestions
    num_heads = trial.suggest_categorical("num_heads", [2, 4, 8, 16, 32])
    num_transformer_blocks = trial.suggest_categorical("num_transformer_blocks", [2, 4, 8, 16, 32])
    learning_rate = trial.suggest_float("learning_rate", 1e-8, 1e-2, log=True)
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "AdamW", "SGD"])
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
    batch_size = trial.suggest_categorical("batch_size", [10, 16, 32])
    label_smoothing = trial.suggest_float("label_smoothing", 0.0, 0.3, step=0.01)
    factor = 1

    print(f"Hyperparameters: num_heads={num_heads}, num_transformer_blocks={num_transformer_blocks}, "
          f"learning_rate={learning_rate}, optimizer={optimizer_name}, weight_decay={weight_decay}, batch_size={batch_size},factor={factor}")
    
    fold_accuracies = []

    for test_fold_idx in range(5):
        
        #clear GPU memory
        torch.cuda.empty_cache()
        
        
        
        test_fold = fold_indices[test_fold_idx]
        remaining_folds = [fold_indices[i] for i in range(5) if i != test_fold_idx]
        val_fold_idx = test_fold_idx % 4
        val_fold = remaining_folds[val_fold_idx]
        train_folds = [fold for fold in remaining_folds if fold != val_fold]

        train_data = np.concatenate([eeg_folds[j] for j in train_folds]).transpose(0, 3, 1, 2)
        train_labels = np.concatenate([labels_folds[j] for j in train_folds])

        val_data = eeg_folds[val_fold].transpose(0, 3, 1, 2)
        val_labels = labels_folds[val_fold]

        test_data = eeg_folds[test_fold].transpose(0, 3, 1, 2)
        test_labels = labels_folds[test_fold]

        balanced_train_data, balanced_train_labels = data_balancer(train_data, train_labels, factor=factor)

        train_dataset = TensorDataset(torch.tensor(balanced_train_data, dtype=torch.float32), 
                                      torch.tensor(balanced_train_labels, dtype=torch.long))
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)

        val_dataset = TensorDataset(torch.tensor(val_data, dtype=torch.float32), 
                                    torch.tensor(val_labels, dtype=torch.long))
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

        test_dataset = TensorDataset(torch.tensor(test_data, dtype=torch.float32), 
                                     torch.tensor(test_labels, dtype=torch.long))
        test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)

        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = TRANS_CNN(input_shape=input_shape, num_transformer_blocks=num_transformer_blocks, 
                          num_heads=num_heads, num_classes=num_classes, embed_dim=128).to(device)

        criterion = nn.CrossEntropyLoss(label_smoothing=label_smoothing)
        optimizer_cls = {"Adam": optim.Adam, "AdamW": optim.AdamW, "SGD": optim.SGD}
        optimizer = optimizer_cls[optimizer_name](model.parameters(), lr=learning_rate, weight_decay=weight_decay)

        early_stopping = EarlyStopping(patience=10)

        epochs = 30
        for epoch in range(epochs):
            model.train()
            for inputs, labels in train_loader:
                inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()

            model.eval()
            val_loss = 0.0
            with torch.no_grad():
                for val_inputs, val_labels in val_loader:
                    val_inputs, val_labels = val_inputs.to(device, non_blocking=True), val_labels.to(device, non_blocking=True)
                    val_outputs = model(val_inputs)
                    loss = criterion(val_outputs, val_labels)
                    val_loss += loss.item()

            val_loss /= len(val_loader)

            early_stopping(val_loss, model)
            if early_stopping.early_stop:
                print(f"Early stopping at epoch {epoch}")
                break

        early_stopping.load_best_model(model)

        model.eval()
        all_preds, all_labels = [], []
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device, non_blocking=True), labels.to(device, non_blocking=True)
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        fold_acc = balanced_accuracy_score(all_labels, all_preds)
        fold_accuracies.append(fold_acc)
        print(f"Trial {trial.number}, Fold {test_fold_idx+1}: Test Accuracy = {fold_acc:.4f}")

        del model
        torch.cuda.empty_cache()

    mean_accuracy = np.mean(fold_accuracies)
    print(f"Trial {trial.number}: Mean Accuracy = {mean_accuracy:.4f}, Fold Accuracies = {fold_accuracies}")
    
    trial.set_user_attr("fold_accuracies", fold_accuracies)
    trial.report(mean_accuracy, step=0)  # Single report after all folds

    if trial.should_prune():
        raise optuna.TrialPruned()
    
    return mean_accuracy

# Start Optuna Study
study = optuna.create_study(
    direction="maximize",
    sampler=optuna.samplers.TPESampler(),
    pruner=optuna.pruners.MedianPruner(n_warmup_steps=5)
)

study.optimize(objective, n_trials=300, n_jobs=3)       

# Best result
print("Best hyperparameters:", study.best_params)


c:\Users\Gabriel\anaconda3\envs\cudaenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[I 2025-04-30 21:27:57,840] A new study created in memory with name: no-name-161658ce-6462-4cc2-8305-bd9f89253656


Hyperparameters: num_heads=32, num_transformer_blocks=16, learning_rate=6.498686732547997e-08, optimizer=AdamW, weight_decay=4.3860827139279903e-05, batch_size=16,factor=1
Hyperparameters: num_heads=2, num_transformer_blocks=32, learning_rate=0.0003817356210826404, optimizer=SGD, weight_decay=3.473507027614272e-05, batch_size=32,factor=1
Hyperparameters: num_heads=32, num_transformer_blocks=8, learning_rate=0.003827867538316409, optimizer=AdamW, weight_decay=5.124889337801125e-06, batch_size=16,factor=1
Divergence detected. Stopping training after 10 epochs.
Early stopping at epoch 10
Trial 2, Fold 1: Test Accuracy = 0.3333
Divergence detected. Stopping training after 10 epochs.
Early stopping at epoch 15
Trial 1, Fold 1: Test Accuracy = 0.3214
Divergence detected. Stopping training after 10 epochs.
Early stopping at epoch 13
Trial 2, Fold 2: Test Accuracy = 0.3333
Divergence detected. Stopping training after 10 epochs.
Early stopping at epoch 27
Trial 0, Fold 1: Test Accuracy = 0.3345

[I 2025-04-30 21:55:32,260] Trial 2 finished with value: 0.3333333333333333 and parameters: {'num_heads': 32, 'num_transformer_blocks': 8, 'learning_rate': 0.003827867538316409, 'optimizer': 'AdamW', 'weight_decay': 5.124889337801125e-06, 'batch_size': 16, 'label_smoothing': 0.25}. Best is trial 2 with value: 0.3333333333333333.


Hyperparameters: num_heads=8, num_transformer_blocks=16, learning_rate=1.2071155153016448e-08, optimizer=SGD, weight_decay=3.65061488613633e-06, batch_size=16,factor=1
Divergence detected. Stopping training after 10 epochs.
Early stopping at epoch 17
Trial 1, Fold 5: Test Accuracy = 0.3258
Trial 1: Mean Accuracy = 0.3329, Fold Accuracies = [np.float64(0.32135397070289823), np.float64(0.3333333333333333), np.float64(0.35062990003542654), np.float64(0.3333333333333333), np.float64(0.32576427138933567)]


[I 2025-04-30 21:58:54,961] Trial 1 finished with value: 0.3328829617588654 and parameters: {'num_heads': 2, 'num_transformer_blocks': 32, 'learning_rate': 0.0003817356210826404, 'optimizer': 'SGD', 'weight_decay': 3.473507027614272e-05, 'batch_size': 32, 'label_smoothing': 0.27}. Best is trial 2 with value: 0.3333333333333333.


...

[I 2025-05-01 23:54:01,709] Trial 138 finished with value: 0.5099554870361385 and parameters: {'num_heads': 16, 'num_transformer_blocks': 4, 'learning_rate': 0.00020904212330170825, 'optimizer': 'Adam', 'weight_decay': 1.0068861174231567e-06, 'batch_size': 10, 'label_smoothing': 0.13}. Best is trial 86 with value: 0.5553539263859596.


Divergence detected. Stopping training after 10 epochs.
Early stopping at epoch 12
Trial 139, Fold 3: Test Accuracy = 0.5589
Divergence detected. Stopping training after 10 epochs.
Early stopping at epoch 26
Trial 139, Fold 4: Test Accuracy = 0.4476
Divergence detected. Stopping training after 10 epochs.
Early stopping at epoch 17
Trial 139, Fold 5: Test Accuracy = 0.5494
Trial 139: Mean Accuracy = 0.5294, Fold Accuracies = [np.float64(0.5029877735197996), np.float64(0.5881503812167034), np.float64(0.5588571846649107), np.float64(0.44757085020242915), np.float64(0.5493960672463922)]


[I 2025-05-02 00:00:26,797] Trial 139 finished with value: 0.529392451370047 and parameters: {'num_heads': 16, 'num_transformer_blocks': 4, 'learning_rate': 5.813158155810114e-05, 'optimizer': 'Adam', 'weight_decay': 0.0007318031906742477, 'batch_size': 10, 'label_smoothing': 0.13}. Best is trial 86 with value: 0.5553539263859596.


RuntimeError: DataLoader worker (pid(s) 28076) exited unexpectedly

: 